In [3]:
import sqlalchemy as sa
import psycopg2
import pandas as pd
import shapely.wkt
import geopandas as gpd

In [140]:
! python -m spacy download en_core_web_lg

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.7/587.7 MB 3.7 MB/s eta 0:00:00m eta 0:00:010:00:02

[notice] A new release of pip available: 22.3 -> 22.3.1
[notice] To update, run: pip install --upgrade pip
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')


In [229]:
! python -m spacy download en_core_web_lm


✘ No compatible package found for 'en_core_web_lm' (spaCy v3.4.3)



In [10]:
import spacy
import pandas as pd
from spacy import displacy
nlp = spacy.load("en_core_web_sm")


In [11]:
df = pd.read_csv("questions.csv")

## Case 1: Shops in Oakland


In [12]:
doc = nlp("Shops in Oakland")
displacy.render(nlp(doc), style="dep", jupyter=True)

## Case 2: Which restraunts are in Oakland?

In [13]:
doc = nlp("Which pubs are located in Oakland?")
displacy.render(doc, style="dep", jupyter=True)

In [14]:
doc = nlp("Where are pubs located in Oakland?")
displacy.render(doc, style="dep", jupyter=True)

In [15]:
doc = nlp("Where are pubs in Oakland?")
displacy.render(doc, style="dep", jupyter=True)

In [16]:
doc = nlp("Where are the pubs in Oakland?")
displacy.render(doc, style="dep", jupyter=True)

In [17]:
doc = nlp("Which pubs are in Oakland?")
displacy.render(doc, style="dep", jupyter=True)

## Case 4: What restraunts are in Oakland?

In [ ]:
doc = nlp("What restraunts are in Oakland")
displacy.render(doc, style="dep", jupyter=True)

In [247]:
doc = nlp("What is the tallest building in Oakland")
displacy.render(doc, style="dep", jupyter=True)

In [18]:
sent = next(doc.sents)
root = sent.root
if root.pos_ == "VERB":
    for child in root.children:
        if child.pos_ == "NOUN" and child.dep_ == "nsubj":
            print(child.text)

## Case: Search for restaurants in Oakland

In [19]:
doc = nlp("Search for restraunts in Oakland")
displacy.render(doc, style="dep", jupyter=True)

In [20]:
sent = next(doc.sents)
root = sent.root
if root.pos_ == "VERB":
    for child in root.children:
        if child.dep_ == "prep":
            for grandchild in child.children:
                if grandchild.pos_ == "NOUN" and grandchild.dep_ == "pobj":
                    print(grandchild.text)


restraunts


# Case: Find me restaurants in Oakland

In [21]:
doc = nlp("Find me restaurants in Oakland")
displacy.render(doc, style="dep", jupyter=True)

In [22]:
doc = nlp("Find me the restaurants in Oakland")
displacy.render(doc, style="dep", jupyter=True)

In [23]:
doc = nlp("Find me all the restaurants in Oakland")
displacy.render(doc, style="dep", jupyter=True)

In [24]:
sent = next(doc.sents)
root = sent.root
if root.pos_ == "VERB":
    for child in root.children:
        if child.dep_ in {"ccomp"} and child.pos_ in {"NOUN", "PROPN"}:
            print(child.text)

In [25]:
doc = nlp("Find me all the restaurants in Oakland")
displacy.render(doc, style="dep", jupyter=True)

In [100]:
NOUN_POS = {"NOUN", "PROPN"}
from spacy.tokens.span import Span
from spacy.tokens.token import Token
from typing import Union

def get_noun_chunk(tok: Token) -> Union[Span, None]:
    for nc in tok.doc.noun_chunks:
        if nc.start <= tok.i < nc.end:
            return nc

# Requires the root of the dependency tree
def get_main_noun(sent: Span) -> Union[Token, None]:
    root = sent.root
    if root.pos_ in NOUN_POS:
        return root
    if root.pos_ == "VERB":
        # "Where are pubs in Oakland"        
        for child in root.children:
            if child.pos_ in NOUN_POS:
                if child.dep_ in {"nsubj", "ccomp", "nsubjpass"}:
                    return child
                elif child.dep_ == "dobj":
                    return child
            elif child.dep_ == "prep":
                if child.dep_ == "pobj" and child.pos_ in NOUN_POS:
                    return child
    # if no noun is found, then use the head of the first noun chunk in the sentence
    for chunk in sent.noun_chunks:
        return chunk.root
    
for row in df.itertuples():
    doc = nlp(row.sentence)
    main_noun = get_main_noun(next(doc.sents))
    if not main_noun:
        print("No main noun found for sentence:", row.sentence)
        continue
    nc = get_noun_chunk(main_noun)
    if not nc:
        print("No noun chunk found", row.main_noun)
    elif (str(nc.text) != row.main_noun):
        print(row.sentence, "|", nc, "|", row.main_noun)

Give me the best restaurants in Oakland! | the best restaurants | best restaurants


In [145]:
doc = nlp("")
displacy.render(doc, style="dep", jupyter=True)
# get_main_noun(next(doc.sents))
sent = next(doc.sents)

StopIteration: 

In [102]:
def get_noun_chunk(tok: Token) -> Union[Span, None]:
    for nc in tok.doc.noun_chunks:
        if nc.start <= tok.i < nc.end:
            return nc

In [103]:
## Identify propositions

doc = nlp("The best restaurants near small parks with quiet seating with many dogs")
displacy.render(doc, style="dep", jupyter=True)
# get_main_noun(next(doc.sents))

sent = first_sentence(doc)


In [ ]:
## Identify propositions

doc = nlp("The best restaurants that have quiet seating")
displacy.render(doc, style="dep", jupyter=True)
# get_main_noun(next(doc.sents))

doc = nlp("The best restaurants seting 100")
displacy.render(doc, style="dep", jupyter=True)


In [ ]:
doc = nlp("Paris restaurants")
displacy.render(doc, style="dep", jupyter=True)

In [ ]:
doc = nlp("Parisian restaurants with outdoor seating")
displacy.render(doc, style="dep", jupyter=True)

In [ ]:
doc = nlp("New York restaurants with outdoor seating")
displacy.render(doc, style="ent", jupyter=True)

In [ ]:
doc = nlp("New York restaurants with outdoor seating")
displacy.render(doc, style="ent", jupyter=True)

- at
- on 
- in 

Disambiguating spatial prepositions: The case of geo-spatial sense detection

- https://onlinelibrary.wiley.com/doi/abs/10.1111/tgis.12976

  - https://figshare.com/s/5ff1f127948145681af5
  - https://figshare.com/s/7407f37544f910fdbb9e
  - https://figshare.com/s/9899575b0617a6a9eaa5

In [189]:
import spacy
from spacy.matcher import Matcher
from spacy.language import Language
from spacy.tokens.doc import Doc

from typing import cast

nlp = spacy.load("en_core_web_lg")
nlp.add_pipe("merge_entities")

matcher = Matcher(nlp.vocab)
matcher.add(
    "IN", [
        [{"LEMMA": {"IN": ["locate", "found"]}, "OP": "?"}, {"DEP": "prep", "LOWER": "in"}],
        [{"LEMMA": {"IN": ["locate", "found"]}, "OP": "?"}, {"DEP": "prep", "LOWER": "within"}],
        [{"LEMMA": {"IN": ["locate", "found"]}, "OP": "?"}, {"DEP": "prep", "LOWER": "inside"}, {"DEP": "prep", "LOWER": "of", "OP": "?"}],
    ]
, greedy="LONGEST")

matcher.add(
    "NOT_IN", [
        [{"DEP": "neg"}, {"LEMMA": {"IN": ["locate", "found"]}, "OP": "?"}, {"DEP": "prep", "LOWER": "in"}],
        [{"DEP": "neg"}, {"LEMMA": {"IN": ["locate", "found"]}, "OP": "?"}, {"DEP": "prep", "LOWER": "within"}],
        [{"DEP": "neg"}, {"LEMMA": {"IN": ["locate", "found"]}, "OP": "?"}, {"DEP": "prep", "LOWER": "inside"}, {"DEP": "prep", "LOWER": "of", "OP": "?"}],
        [{"LEMMA": "outside", "DEP": "prep"}, {"DEP": "prep", "LOWER": "of", "OP": "?"}],
    ]
, greedy="LONGEST")


def first_sentence(doc):
    return next(doc.sents)

sentences = [
    "San Francisco shops",
    "Shops in San Francisco",
    "Find shops in San Francisco",
    "Find shops located in San Francisco",
    "Give me shops in San Francisco",
    "Give me shops located in San Francisco",
    "Search for shops in San Francisco",
    "Search for shops located in San Francisco",
    "What shops are located in San Francisco",
    "What shops are in San Francisco",
    "Where are shops in San Francisco",
    "Which shops are in San Francisco",
    "Which shops are located in San Francisco",
    "Locate shops in San Francisco",
    "Please find shops in San Francisco",
    "Select shops in San Francisco",
    "Select for shops in San Francisco",
    "Search in San Francisco for shops",
    "Where are the shops in San Francisco",
    "Where are shops in San Francisco",
    "Where is a shop in San Francisco",
    "Which pubs are located in Oakland?",
    "Not in 5 miles of Oakland",
    "shops within 2 hours and 10 minutes of Oakland",
    "The best restaurants that have quiet seating",
    "Shops not in San Francisco",
    "Shops outside of San Francisco",
    "Shops not inside of San Francisco",
    "Shops that are not inside of San Francisco",
    "Shops not located in San Francisco",
    "Shops not included in San Francisco",
    "Shops that are not included in San Francisco",
    "Shops that are outside of San Francisco",
    "Give me the best restaurants in Oakland!",
    "Find the tallest mountain in Idaho."
    "San Francisco Chinese restaurants",
    "Chinese restaurants",
    "San Francisco restaurants",
    "Chinese San Francisco restaurants",
    "San Francisco Chinese restaurants",
    "Chinese restaurants in San Francisco",
    "Irish pubs in San Francisco",
]

def noun_chunk_type(span: Span) -> str:
    # Assuming that all entities have been merged into a single token
    print("Span", span.root)
    if len(span) == 1 and span.root.ent_type_ == "GPE":
        return "NAMED_SPATIAL_ENTITY"
    else:
        return "SPATIAL_FEATURE"
    
def noun_chunk_cleaner(span: Span):
    # Assuming that all entities have been merged into a single token
    nouns = [t for t in span if t.pos_ in {"NOUN", "PROPN"}]
    if len(nouns) == 0:
        return None
    if len(nouns) == 1 and nouns[0].ent_type_ == "GPE":
        return Span(span.doc, nouns[0].i, nouns[0].i + 1)
    return span


def processor(doc):
    sent = first_sentence(doc)
    main_noun = get_main_noun(sent)
    if main_noun:
        loc_subj = get_noun_chunk(main_noun)
        if not loc_subj:
            raise ValueError("No noun chunk found for main noun")    
        loc_subj = noun_chunk_cleaner(loc_subj)    
        if loc_subj is None:
            raise ValueError("No noun chunk found for main noun")
    else:
        raise ValueError("No main noun chunk found")

    slots = [
        {"start": loc_subj.start, "end": loc_subj.end - 1,
         "type": noun_chunk_type(loc_subj),
         "label": "SUBJECT"}
    ]

    # Check for a spatial relation preposition
    intent = None
    matches = list(matcher(doc))
    if matches:
        # reverse because we want to replace the earliest matches first
        matches.sort(key=lambda x: x[1], reverse=True)
        filtered_matches = []
        while matches:
            m = matches.pop()
            if filtered_matches and m[1] <= filtered_matches[-1][2]:
                continue
            filtered_matches.append(m)
        for m_id, m_start, m_end in filtered_matches[:1]:
            prep = doc[m_end - 1]
            pobj = None
            for c in prep.children:
                if c.dep_ == "pobj":
                    pobj = c
            if pobj:
                pobj_span = Span(doc, pobj.left_edge.i, pobj.right_edge.i + 1)
                intent = doc.vocab.strings[m_id]
                pboj_span = cast(Span, noun_chunk_cleaner(pobj_span))
                if pobj_span:                    
                    slots.append({"start": pboj_span.start, "end": pboj_span.end - 1,
                                "type": noun_chunk_type(pobj_span),
                                "label": "OBJECT"})
    else:
        intent = "SIMPLE"
                
    return {
        "words": [str(t) for t in sent],
        "text": str(doc),
        "slots": slots,
        "intent": intent
    }
 
parsed_sents = []   
for sent in sentences:
    doc = nlp(sent)
    parsed = processor(doc)
    data = {
        "text": sent,
        "intent": parsed["intent"],
        "slots": [doc[s["start"]:s["end"] + 1].text for s in parsed["slots"]]
    }
    parsed_sents.append(data)
    
from ruamel.yaml import YAML
yaml = YAML()
with open("parsed_sents.yml", "w") as f:
    yaml.dump(parsed_sents, f)

Span shops
Span Shops
Span San Francisco
Span shops
Span San Francisco
Span shops
Span San Francisco
Span shops
Span San Francisco
Span shops
Span San Francisco
Span Search
Span San Francisco
Span Search
Span San Francisco
Span shops
Span San Francisco
Span shops
Span San Francisco
Span shops
Span San Francisco
Span shops
Span San Francisco
Span shops
Span San Francisco
Span shops
Span San Francisco
Span shops
Span San Francisco
Span shops
Span San Francisco
Span shops
Span San Francisco
Span Search
Span San Francisco
Span shops
Span San Francisco
Span shops
Span San Francisco
Span shop
Span San Francisco
Span pubs
Span Oakland
Span 5 miles
Span shops
Span 2 hours and 10 minutes
Span restaurants
Span Shops
Span San Francisco
Span Shops
Span Shops
Span Shops
Span Shops
Span San Francisco
Span Shops
Span San Francisco
Span Shops
Span San Francisco
Span Shops
Span San Francisco
Span restaurants
Span Oakland
Span mountain
Span Idaho
Span restaurants
Span restaurants
Span restaurants
Span r

## Chinese restaurants vs. San Francisco retaurants

In [211]:
doc = nlp("Chinese restaurant")
doc[0].ent_type_


Degree=Pos

In [196]:
displacy.render(doc, style="dep")

In [197]:
displacy.render(doc, style="ent")

In [198]:
doc = nlp("Chinese restaurant")

In [203]:
doc = nlp("San Francisco Italian restaurant")

In [204]:
displacy.render(doc, style="dep")

In [206]:
doc = nlp("Italian San Francisco restaurant")
displacy.render(doc, style="dep")

In [210]:
# Noun chunk which does not have a GPE as its root
nc = get_noun_chunk(doc[0])
root = nc.root
root.ent_type_


''

In [176]:
sent_df = []
for i, sent in enumerate(sentences):
    doc = nlp(sent)
    df = pd.DataFrame({
        "sent_id": [i for _ in doc],
        "word_id": [t.i for t in doc],
        "orth": [t.orth_ for t in doc],
        "spaces": [bool(len(t.whitespace_)) for t in doc],
        "tags": [t.tag_ for t in doc],
        "pos": [t.pos_ for t in doc],
        "sent_starts": [t.is_sent_start for t in doc],
        "entities": [t.ent_type_ for t in doc],
        "heads": [t.head.i for t in doc],
        "deps": [t.dep_ for t in doc],
        "morphs": [t.morph for t in doc],
        "lemma": [t.lemma_ for t in doc]
    })
    sent_df.append(df)



In [106]:
# "within", "in", "inside", "is located in", "is included in"
from spacy.matcher import DependencyMatcher

matcher = DependencyMatcher(nlp.vocab)
patterns_1 = [
    [
        {
            "RIGHT_ID": "verb", 
            "RIGHT_ATTRS": {"POS": "VERB", "LOWER": word}
        },
        {
            "LEFT_ID": "verb",
            "RIGHT_ID": "prep",
            "REL_OP": ">",
            "RIGHT_ATTRS": {"POS": "ADP", "LOWER": "in", "DEP": "prep"}
        }
    ] for word in ["located", "included"]
]

patterns_2 = [[
    {"RIGHT_ID": "prep", 
     "RIGHT_ATTRS": 
         {"DEP": "prep", "LOWER": word}
    },
] for word in ["in", "within", "inside"]]


matcher.add("IN", [*patterns_1])

for s in sentences:
    doc = nlp(s)
    matches = matcher(doc)
    print(s)
    print(matches)


San Francisco shops
[]
Shops in San Francisco
[]
Find shops in San Francisco
[]
Find shops located in San Francisco
[(1292078113972184607, [2, 3])]
Give me shops in San Francisco
[]
Give me shops located in San Francisco
[(1292078113972184607, [3, 4])]
Search for shops in San Francisco
[]
Search for shops located in San Francisco
[(1292078113972184607, [3, 4])]
What shops are located in San Francisco
[(1292078113972184607, [3, 4])]
What shops are in San Francisco
[]
Where are shops in San Francisco
[]
Which shops are in San Francisco
[]
Which shops are located in San Francisco
[(1292078113972184607, [3, 4])]
Locate shops in San Francisco
[]
Please find shops in San Francisco
[]
Select shops in San Francisco
[]
Select for shops in San Francisco
[]
Search in San Francisco for shops
[]
Where are the shops in San Francisco
[]
Where are shops in San Francisco
[]
Where is a shop in San Francisco
[]
Which pubs are located in Oakland?
[(1292078113972184607, [3, 4])]
Not in 5 miles of Oakland
[

In [143]:
displacy.render(nlp("shops within 2 hours and 10 minutes of Oakland"), style="ent")

In [278]:
# "outside", "not in", "not inside", "not located in", "not included in"
matcher = DependencyMatcher(nlp.vocab)
patterns_1 = [
    [
        {
            "RIGHT_ID": "verb", 
            "RIGHT_ATTRS": {"POS": "VERB", "LOWER": word}
        },
        {
            "LEFT_ID": "verb",
            "RIGHT_ID": "prep",
            "REL_OP": ">",
            "RIGHT_ATTRS": {"POS": "ADP", "LOWER": 'in', "DEP": "prep"}
        },
        {
            "LEFT_ID": "verb",
            "RIGHT_ID": "neg",
            "REL_OP": ">",
            "RIGHT_ATTRS": {"DEP": "neg"}
        }        
    ] for word in ["located", "included"]
]

patterns_2 = [[
    {"RIGHT_ID": "prep", 
     "RIGHT_ATTRS": {"DEP": "prep", "LOWER": word}},
    {
        "LEFT_ID": "prep",
        "RIGHT_ID": "neg",
        "REL_OP": ">",
        "RIGHT_ATTRS": {"DEP": "neg"}
    }         
] for word in ["in", "within", "inside"]]

patterns_3 = [[
    {
        "RIGHT_ID": "prep", 
        "RIGHT_ATTRS": {"DEP": "prep", "LOWER": word}
    },       
] for word in ["outside"]]

patterns_3 = [[
    {
        "RIGHT_ID": "prep", 
        "RIGHT_ATTRS": {"DEP": "prep", "LOWER": word}
    },
    {
        "LEFT_ID": "prep",
        "RIGHT_ID": "prep2", 
        "REL_OP": ">",        
        "RIGHT_ATTRS": {"DEP": "prep", "LOWER": "of"}
    },            
] for word in ["outside"]]

patterns_4 = [[
    {"RIGHT_ID": "prep", 
     "RIGHT_ATTRS": {"LOWER": word, "POS": "ADV"}
     },
    {
        "LEFT_ID": "prep",
        "RIGHT_ID": "neg",
        "REL_OP": ">",
        "RIGHT_ATTRS": {"DEP": "neg"}
    },  
    {
        "LEFT_ID": "prep",
        "RIGHT_ID": "prep2",
        "REL_OP": ">",
        "RIGHT_ATTRS": {"DEP": "prep", "LOWER": "of"}
    }         
] for word in ["inside"]]

patterns_5 = [[
    {"RIGHT_ID": "prep", 
     "RIGHT_ATTRS": {"LOWER": word, "POS": "ADV"}
    },
    {
        "LEFT_ID": "prep",
        "RIGHT_ID": "neg",
        "REL_OP": ">",
        "RIGHT_ATTRS": {"DEP": "neg"}
    },  
    {
        "LEFT_ID": "prep",
        "RIGHT_ID": "prep2",
        "REL_OP": ">",
        "RIGHT_ATTRS": {"DEP": "prep", "LOWER": "of"}
    },
    {
        "LEFT_ID": "prep",
        "RIGHT_ID": "prep2",
        "REL_OP": ">",
        "RIGHT_ATTRS": {"DEP": "prep", "LOWER": "of"}
    }           
] for word in ["inside"]]

matcher.add("IN", [*patterns_1, *patterns_2, *patterns_3, *patterns_4, *patterns_5])

for s in sentences:
    doc = nlp(s)
    matches = matcher(doc)
    print(matches)


ValueError: [E101] RIGHT_ID should be a new node and LEFT_ID should already have have been declared in previous edges.

In [124]:
# "outside", "not in", "not inside", "not located in", "not included in"
matcher = DependencyMatcher(nlp.vocab)
patterns_1 = [
    [
        {
            "RIGHT_ID": "verb", 
            "RIGHT_ATTRS": {"POS": "VERB", "LOWER": word}
        },
        {
            "LEFT_ID": "verb",
            "RIGHT_ID": "prep",
            "REL_OP": ">",
            "RIGHT_ATTRS": {"POS": "ADP", "LOWER": 'in', "DEP": "prep"}
        },
        {
            "LEFT_ID": "verb",
            "RIGHT_ID": "neg",
            "REL_OP": ">",
            "RIGHT_ATTRS": {"DEP": "neg"}
        }        
    ] for word in ["located", "included"]
]

patterns_2 = [[
    {"RIGHT_ID": "prep", 
     "RIGHT_ATTRS": {"DEP": "prep", "LOWER": word}},
    {
        "LEFT_ID": "prep",
        "RIGHT_ID": "neg",
        "REL_OP": ">",
        "RIGHT_ATTRS": {"DEP": "neg"}
    }         
] for word in ["in", "within", "inside"]]

patterns_3 = [[
    {
        "RIGHT_ID": "prep", 
        "RIGHT_ATTRS": {"DEP": "prep", "LOWER": word}
    },       
] for word in ["outside"]]

patterns_3 = [[
    {
        "RIGHT_ID": "prep", 
        "RIGHT_ATTRS": {"DEP": "prep", "LOWER": word}
    },
    {
        "LEFT_ID": "prep",
        "RIGHT_ID": "prep2", 
        "REL_OP": ">",        
        "RIGHT_ATTRS": {"DEP": "prep", "LOWER": "of"}
    },            
] for word in ["outside"]]

patterns_4 = [[
    {"RIGHT_ID": "prep", 
     "RIGHT_ATTRS": {"LOWER": word, "POS": "ADV"}
     },
    {
        "LEFT_ID": "prep",
        "RIGHT_ID": "neg",
        "REL_OP": ">",
        "RIGHT_ATTRS": {"DEP": "neg"}
    },  
    {
        "LEFT_ID": "prep",
        "RIGHT_ID": "prep2",
        "REL_OP": ">",
        "RIGHT_ATTRS": {"DEP": "prep", "LOWER": "of"}
    }         
] for word in ["inside"]]

patterns_5 = [[
    {"RIGHT_ID": "prep", 
     "RIGHT_ATTRS": {"LOWER": word, "POS": "ADV"}
    },
    {
        "LEFT_ID": "prep",
        "RIGHT_ID": "neg",
        "REL_OP": ">",
        "RIGHT_ATTRS": {"DEP": "neg"}
    },  
    {
        "LEFT_ID": "prep",
        "RIGHT_ID": "prep2",
        "REL_OP": ">",
        "RIGHT_ATTRS": {"DEP": "prep", "LOWER": "of"}
    },
    {
        "LEFT_ID": "prep",
        "RIGHT_ID": "prep2",
        "REL_OP": ">",
        "RIGHT_ATTRS": {"DEP": "prep", "LOWER": "of"}
    }           
] for word in ["inside"]]

matcher.add("IN", [*patterns_1, *patterns_2, *patterns_3, *patterns_4, *patterns_5])

for s in sentences:
    doc = nlp(s)
    matches = matcher(doc)
    print(s)
    print(matches)


ValueError: [E101] RIGHT_ID should be a new node and LEFT_ID should already have have been declared in previous edges.

In [ ]:
sentences = [
    "Shops not in San Francisco",
    "Shops outside of San Francisco",
    "Shops not inside of San Francisco",
    "Shops that are not inside of San Francisco",
    "Shops not located in San Francisco",
    "Shops not included in San Francisco",
    "Shops that are not included in San Francisco",
    "Shops that are outside of San Francisco",
]
for s in sentences:
    doc = nlp(s)
    print(doc)

In [126]:
sentences = [
    "Shops within 5 miles of San Francisco",
    "Shops at most 5 miles from San Francisco",
    "Shops that are within 5 miles of San Francisco",
    "Shops that are located within in 5 miles of San Francisco",
    "Shops less than 5 miles from San Francisco",    
]
for s in sentences:
    doc = nlp(s)
    for tok in doc:
        if tok.dep_ == "prep":
            print(tok.text, tok.dep_, tok.head.text, "|", doc[tok.left_edge.i:tok.right_edge.i+1])


within prep Shops | within 5 miles of San Francisco
of prep 5 miles | of San Francisco
at prep Shops | at most 5 miles from San Francisco
from prep 5 miles | from San Francisco
within prep are | within 5 miles of San Francisco
of prep 5 miles | of San Francisco
within prep located | within
in prep located | in 5 miles of San Francisco
of prep 5 miles | of San Francisco
from prep miles | from San Francisco


In [127]:
sentences = [
    "Shops within 5 miles of San Francisco",
    "Shops not within 5 miles of San Francisco",
    "Shops at most 5 miles from San Francisco",
    "Shops at least 5 miles from San Francisco",
    "Shops not within 5 miles from San Francisco",
    "Shops that are within 5 miles of San Francisco",
    "Shops that are located within in 5 miles of San Francisco",
    "Shops less than 5 miles from San Francisco",    
    "Shops more than 5 miles from San Francisco",    
]
for s in sentences:
    doc = nlp(s)
    displacy.render(doc, style="dep", jupyter=True)


In [162]:
sentences = [
    "Shops within 5 hours of San Francisco",
    "Shops not within 5 minute of San Francisco",
    "Shops at most 5 hours and 10 minutes from San Francisco",
]
for s in sentences:
    doc = nlp(s)
    displacy.render(doc, style="dep", jupyter=True)


In [128]:
sentences = [
    "Shops within 5 hours of San Francisco",
    "Shops not within 5 minute of San Francisco",
    "Shops at most 5 hours and 10 minutes from San Francisco",
    "Shops more than 5:00 from San Francisco",
    "Shops within 5 hours of San Francisco",
    "Shops not within 5 minute of San Francisco",
    "Shops at most 5 hours and 10 minutes from San Francisco"
]
for s in sentences:
    doc = nlp(s)
    displacy.render(doc, style="dep", jupyter=True)


In [132]:
from spacy.matcher import Matcher
matcher = Matcher(nlp.vocab)

# Within
patterns = [
    [
        *subpat,
        {"ENT_TYPE": "TIME", "OP": "+"},
        {"POS": "ADP", "dep": "prep", "OP": "?"}
    ]
    for subpat in [
        [
            {"LEMMA": "less", "POS": "ADP"},
            {"LEMMA": "than", "POS": "ADP"}
        ],
        [
            {"LEMMA": "within", "POS": "ADP"}
        ],
        [
            {"LEMMA": "at", "POS": "ADP"},
            {"LEMMA": "most"}
        ],
        [
            {"LEMMA": "not"},
            {"LEMMA": "more"},
            {"LEMMA": "than"}
        ],        
    ]        
]

matcher.add("LESS_THAN_TIME", patterns, greedy="LONGEST")

patterns = [
    [
        *subpat,
        {"ENT_TYPE": "TIME", "OP": "+"},
        {"POS": "ADP", "dep": "prep", "OP": "?"}
    ]
    for subpat in [
        [
            {"LEMMA": "more"},
            {"LEMMA": "than"}
        ],
        [
            {"LEMMA": "at"},
            {"LEMMA": "least"}
        ],        
        [
            {"LEMMA": "not", "DEP": "neg"},
            {"LEMMA": "within", "POS": "ADP"},
        ],
        [
            {"LEMMA": "not", "DEP": "neg"},
            {"LEMMA": "less", "POS": "ADP"},
            {"LEMMA": "than", "POS": "ADP"}
        ],               
    ]        
]

matcher.add("MORE_THAN_TIME", patterns, greedy="LONGEST")

for s in sentences:
    doc = nlp(s)
    matches = matcher(doc)
    print(s)
    for m in matches:
        print(doc.vocab.strings[m[0]], doc[m[1]:m[2]])
    print()

Shops within 5 hours of San Francisco
LESS_THAN_TIME within 5 hours of

Shops not within 5 minute of San Francisco
MORE_THAN_TIME not within 5 minute of
LESS_THAN_TIME within 5 minute of

Shops at most 5 hours and 10 minutes from San Francisco

Shops more than 5:00 from San Francisco
MORE_THAN_TIME more than 5:00 from

Shops within 5 hours of San Francisco
LESS_THAN_TIME within 5 hours of

Shops not within 5 minute of San Francisco
MORE_THAN_TIME not within 5 minute of
LESS_THAN_TIME within 5 minute of

Shops at most 5 hours and 10 minutes from San Francisco



In [133]:
time_examples = [
"5 hours and 10 minutes",
"10 minutes",
"5:00",
"1:20"
]
import ctparse
for t in time_examples:
    parsed = ctparse.ctparse(t, latent_time=True)
    print(t, "|", parsed)

5 hours and 10 minutes | 10 minutes s=-797.267 p=(138, 137, 138, 'ruleDigitDuration', 'ruleDigitDuration')
10 minutes | 10 minutes s=-4.227 p=(138, 'ruleDigitDuration')
5:00 | 2022-11-23 05:00 (X/X) s=-1.001 p=(129, 'ruleHHMM')
1:20 | 2022-11-23 01:20 (X/X) s=-1.001 p=(129, 'ruleHHMM')
